# 🚀 NeuroTrader PPO Training Farm (Batch Mode)

Train **multiple PPO models** with randomized hyperparameters in a single run.

## Features:
- 🎲 Random hyperparameters for each model
- ⚡ SubprocVecEnv for parallel environments (better GPU utilization)
- 📊 Auto-evaluation and ranking
- 💾 All models saved to Google Drive

---

## Before Running:
1. **Runtime → Change runtime type → GPU (T4)**
2. Have ~500MB+ free on Google Drive

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install stable-baselines3[extra] gymnasium pandas pyarrow -q

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Imports
import os
import json
import time
import shutil
import numpy as np
import pandas as pd
import torch
from datetime import datetime

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
import gymnasium as gym
from gymnasium import spaces

# Check GPU
print(f"✅ Setup complete!")
print(f"   PyTorch: {torch.__version__}")
print(f"   GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
OUTPUT_DIR = '/content/drive/MyDrive/NeuroTrader_Models'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Output directory: {OUTPUT_DIR}")

## 2. Upload Training Data

In [ ]:
from google.colab import files

# Create data directory
os.makedirs('data/raw', exist_ok=True)

# Upload file
print("📤 Please upload: XAUUSDm_M5_raw.parquet")
uploaded = files.upload()

# Move to data folder
for filename in uploaded.keys():
    shutil.move(filename, f'data/raw/{filename}')
    print(f"✅ Saved to: data/raw/{filename}")

## 3. Training Configuration

Adjust the sliders below to customize your training run.

In [ ]:
#@title ⚙️ Training Settings

NUM_MODELS = 5  #@param {type:"slider", min:1, max:10, step:1}
STEPS_PER_MODEL = 300000  #@param {type:"slider", min:100000, max:1000000, step:100000}
NUM_PARALLEL_ENVS = 4  #@param {type:"slider", min:1, max:8, step:1}

print("📋 Training Plan:")
print(f"   Models to train: {NUM_MODELS}")
print(f"   Steps per model: {STEPS_PER_MODEL:,}")
print(f"   Parallel envs:   {NUM_PARALLEL_ENVS}")
print(f"   Total steps:     {NUM_MODELS * STEPS_PER_MODEL:,}")

## 4. Define Trading Environment

In [ ]:
class TradingEnv(gym.Env):
    """
    Simple Trading Environment for PPO training.
    Actions: 0=HOLD, 1=BUY, 2=SELL
    """
    
    def __init__(self, df, lookback=20):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.lookback = lookback
        self.current_step = lookback
        
        # Action space: HOLD, BUY, SELL
        self.action_space = spaces.Discrete(3)
        
        # Observation space: 22 features
        self.observation_space = spaces.Box(
            low=-np.inf, 
            high=np.inf, 
            shape=(22,), 
            dtype=np.float32
        )
        
        # Trading state
        self.position = 0  # 0=flat, 1=long
        self.entry_price = 0
        self.balance = 10000
        self.equity = 10000
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.lookback
        self.position = 0
        self.entry_price = 0
        self.balance = 10000
        self.equity = 10000
        return self._get_observation(), {}
    
    def _get_observation(self):
        """Calculate observation from price data."""
        window = self.df.iloc[self.current_step - self.lookback : self.current_step]
        close = window['close'].values
        
        # Calculate returns
        returns = np.diff(close) / (close[:-1] + 1e-8)
        
        # Build observation vector
        observation = np.concatenate([
            returns[-10:],  # Last 10 returns
            [close[-1] / close[0] - 1],  # Period return
            [(close[-1] - close.min()) / (close.max() - close.min() + 1e-8)],  # Price position
            [np.std(returns)],  # Volatility
            [self.position],  # Current position (0 or 1)
            [self.equity / 10000 - 1],  # PnL percentage
            np.zeros(7)  # Padding for 22 features
        ])
        
        return observation.astype(np.float32)
    
    def step(self, action):
        """Execute one trading step."""
        current_price = self.df.iloc[self.current_step]['close']
        reward = 0
        
        # Execute action
        if action == 1 and self.position == 0:  # BUY
            self.position = 1
            self.entry_price = current_price
            reward = 0.01  # Small entry bonus
            
        elif action == 2 and self.position == 1:  # SELL
            pnl_pct = (current_price - self.entry_price) / self.entry_price
            self.balance *= (1 + pnl_pct)
            reward = pnl_pct * 100  # Scaled for learning
            self.position = 0
            self.entry_price = 0
        
        # Update equity
        if self.position == 1:
            unrealized_pnl = (current_price - self.entry_price) / self.entry_price
            self.equity = self.balance * (1 + unrealized_pnl)
        else:
            self.equity = self.balance
        
        # Move to next step
        self.current_step += 1
        
        # Check termination conditions
        done = self.current_step >= len(self.df) - 1
        truncated = self.equity < 5000  # Circuit breaker at -50%
        
        return self._get_observation(), reward, done, truncated, {'equity': self.equity}


print("✅ TradingEnv defined!")

## 5. Load Training Data

In [ ]:
# Load data
DATA_PATH = 'data/raw/XAUUSDm_M5_raw.parquet'
DF = pd.read_parquet(DATA_PATH)

# Rename column if needed
if 'tick_volume' in DF.columns:
    DF.rename(columns={'tick_volume': 'volume'}, inplace=True)

print(f"✅ Loaded data: {len(DF):,} rows")
print(f"   Date range: {DF.index[0]} to {DF.index[-1]}")

## 6. Random Hyperparameter Generator

In [ ]:
def generate_random_config():
    """
    Generate randomized hyperparameters for model diversity.
    Each call produces a unique configuration.
    """
    config = {
        'learning_rate': np.random.choice([1e-4, 3e-4, 5e-4, 7e-4, 1e-3]),
        'n_steps': int(np.random.choice([512, 1024, 2048, 4096])),
        'batch_size': int(np.random.choice([32, 64, 128, 256])),
        'ent_coef': round(np.random.uniform(0.01, 0.1), 4),
        'gamma': np.random.choice([0.95, 0.97, 0.99, 0.995]),
        'gae_lambda': round(np.random.uniform(0.9, 0.99), 2),
        'clip_range': np.random.choice([0.1, 0.2, 0.3]),
        'n_epochs': int(np.random.choice([5, 10, 15, 20])),
    }
    return config

# Test the generator
print("🎲 Example random config:")
for key, value in generate_random_config().items():
    print(f"   {key}: {value}")

## 7. Batch Training 🚀

This cell trains multiple models with different random configurations.

In [ ]:
# Function to create environment (needed for SubprocVecEnv)
def make_env():
    return TradingEnv(DF)

# Store results
training_results = []

# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"\n{'='*60}")
print(f"🚀 STARTING BATCH TRAINING")
print(f"   Models: {NUM_MODELS}")
print(f"   Steps each: {STEPS_PER_MODEL:,}")
print(f"   Device: {device.upper()}")
print(f"{'='*60}\n")

total_start_time = time.time()

for model_idx in range(NUM_MODELS):
    # Generate unique seed based on current time
    seed = int(time.time() * 1000) % 1000000
    np.random.seed(seed)
    
    # Generate random config
    config = generate_random_config()
    
    # Create parallel environments
    env = SubprocVecEnv([make_env for _ in range(NUM_PARALLEL_ENVS)])
    
    # Create PPO model with random hyperparameters
    model = PPO(
        "MlpPolicy",
        env,
        device=device,
        verbose=0,  # Quiet mode
        seed=seed,
        learning_rate=config['learning_rate'],
        n_steps=config['n_steps'],
        batch_size=config['batch_size'],
        ent_coef=config['ent_coef'],
        gamma=config['gamma'],
        gae_lambda=config['gae_lambda'],
        clip_range=config['clip_range'],
        n_epochs=config['n_epochs']
    )
    
    # Train
    print(f"[{model_idx+1}/{NUM_MODELS}] Training... (lr={config['learning_rate']:.0e}, ent={config['ent_coef']:.3f})")
    train_start = time.time()
    model.learn(total_timesteps=STEPS_PER_MODEL, progress_bar=True)
    train_duration = (time.time() - train_start) / 60
    
    # Quick evaluation on test data (last 5000 rows)
    test_env = TradingEnv(DF[-5000:])
    obs, _ = test_env.reset()
    total_trades = 0
    
    for _ in range(4000):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = test_env.step(action)
        if action in [1, 2]:
            total_trades += 1
        if done or truncated:
            break
    
    final_return = (info['equity'] / 10000 - 1) * 100
    
    # Save model
    model_name = f"ppo_{model_idx+1:02d}_s{seed}"
    save_path = f"{OUTPUT_DIR}/{model_name}"
    model.save(save_path)
    
    # Store result
    result = {
        'name': model_name,
        'seed': seed,
        'return_pct': round(final_return, 2),
        'trades': total_trades,
        'duration_min': round(train_duration, 1),
        'config': config
    }
    training_results.append(result)
    
    # Cleanup
    env.close()
    
    # Print result
    print(f"    ✅ {model_name}: {final_return:+.1f}% | {total_trades} trades | {train_duration:.1f}min\n")

# Calculate total time
total_duration = (time.time() - total_start_time) / 60

# Save training log
log_path = f"{OUTPUT_DIR}/batch_training_log.json"
with open(log_path, 'w') as f:
    json.dump({
        'timestamp': datetime.now().isoformat(),
        'total_models': NUM_MODELS,
        'steps_per_model': STEPS_PER_MODEL,
        'total_duration_min': round(total_duration, 1),
        'results': training_results
    }, f, indent=2)

print(f"{'='*60}")
print(f"🎉 BATCH TRAINING COMPLETE!")
print(f"   Total time: {total_duration:.1f} minutes")
print(f"   Models saved: {len(training_results)}")
print(f"   Log file: {log_path}")
print(f"{'='*60}")

## 8. Results Summary

In [ ]:
# Sort by return (best first)
sorted_results = sorted(training_results, key=lambda x: x['return_pct'], reverse=True)

print("\n📊 Results Ranking (Best to Worst):")
print("=" * 50)
for rank, result in enumerate(sorted_results, 1):
    emoji = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
    print(f"{emoji} #{rank} {result['name']}: {result['return_pct']:+.1f}% ({result['trades']} trades)")

# Best model info
best = sorted_results[0]
print(f"\n🏆 Best Model: {best['name']}")
print(f"   Return: {best['return_pct']:+.1f}%")
print(f"   Trades: {best['trades']}")
print(f"   Config: lr={best['config']['learning_rate']:.0e}, ent={best['config']['ent_coef']:.3f}, gamma={best['config']['gamma']}")

## 9. List Saved Files

In [ ]:
print("📁 Files in output directory:")
print("-" * 40)
for filename in sorted(os.listdir(OUTPUT_DIR)):
    filepath = f"{OUTPUT_DIR}/{filename}"
    size_mb = os.path.getsize(filepath) / 1024 / 1024
    print(f"   {filename} ({size_mb:.2f} MB)")

print(f"\n💡 To use these models:")
print(f"   1. Download from Google Drive: MyDrive/NeuroTrader_Models/")
print(f"   2. Copy to Desktop: NeuroTrader/models/farm/")
print(f"   3. Run: python scripts/replay_ghosts.py --source models/farm/")